In [10]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [11]:
import pandas as pd
from fpdf import FPDF

In [12]:
from pydantic import BaseModel,Field
from typing import List

In [13]:
from utils.summarise import summarize_dataframe

In [29]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableLambda

import random
import json

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [16]:
load_dotenv()

True

In [17]:
customers_data_summary = pd.read_json("../data/customers_messy_data.json")
products_data_summary = pd.read_json("../data/products_inconsistent_data.json")
orders_data_summary = pd.read_csv("../data/orders_unstructured_data.csv")

In [18]:
summarized_customers_data = summarize_dataframe(customers_data_summary,"customers")
summarized_products_data = summarize_dataframe(products_data_summary,"products")
summarized_orders_data = summarize_dataframe(orders_data_summary,"orders")

In [19]:
# for col_summary in summarized_customers_data:
#     print(f"Dataset       : {col_summary['dataset']}")
#     print(f"Column        : {col_summary['column']}")
#     print(f"Types         : {', '.join(col_summary['types'])}")
#     print(f"Sample Values : {col_summary['sample_values']}")
#     print(f"Unique Count  : {col_summary['unique_count']}")
#     print(f"Null %        : {col_summary['null_percentage']}%")
#     print(f"Notes         : {col_summary['notes'] if col_summary['notes'] else 'None'}")
#     print("-" * 60)

In [20]:
# for col_summary in summarized_products_data:
#     print(f"Dataset       : {col_summary['dataset']}")
#     print(f"Column        : {col_summary['column']}")
#     print(f"Types         : {', '.join(col_summary['types'])}")
#     print(f"Sample Values : {col_summary['sample_values']}")
#     print(f"Unique Count  : {col_summary['unique_count']}")
#     print(f"Null %        : {col_summary['null_percentage']}%")
#     print(f"Notes         : {col_summary['notes'] if col_summary['notes'] else 'None'}")
#     print("-" * 60)

In [21]:
summary_df = pd.DataFrame(summarized_orders_data)

In [22]:
class IssueOutput(BaseModel):
    issues: List[str]



parser = PydanticOutputParser(pydantic_object=IssueOutput)

In [31]:
def generate_data_issue_report(dataset_name, summary_text, sample_records_text):
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

    # Prompt template
    issue_detection_template = """
        You are a data quality analyst. Given the following column-wise summary and a few sample entries from a dataset.
        
        {format_instructions}

        Your task is to analyze the dataset and identify potential data quality issues. You should
        identify as many real *data quality issues* as possible, such as:

        - Duplicate or redundant columns
        - Mixed data types
        - Missing or null values
        - Inconsistent formatting or casing
        - Invalid entries
        - Other structural anomalies

        ### Dataset Name: {dataset_name}

        ### Column-wise Summary:
        {summary}

        ### Few Sample Records from the Dataset:
        {samples}

        Do not assume structure that is not shown.
        """
    
    prompt1 = PromptTemplate.from_template(issue_detection_template,partial_variables={"format_instructions": parser.get_format_instructions()})
    # chain1 = LLMChain(llm=llm, prompt=prompt1, output_parser=parser) 
    chain1 = prompt1 | llm | parser

    # Run chain
    return chain1.invoke({
        "dataset_name": dataset_name,
        "summary": summary_text,
        "samples": sample_records_text
    })


In [24]:
def format_summary_text(summarized_data):
    output = []
    for col_summary in summarized_data:
        output.append(f"Column        : {col_summary['column']}")
        output.append(f"Types         : {', '.join(col_summary['types'])}")
        output.append(f"Sample Values : {col_summary['sample_values']}")
        output.append(f"Unique Count  : {col_summary['unique_count']}")
        output.append(f"Null %        : {col_summary['null_percentage']}%")
        output.append(f"Notes         : {col_summary['notes'] if col_summary['notes'] else 'None'}")
        output.append("-" * 40)
    return "\n".join(output)

In [25]:
def get_random_samples(file_path, n=10, is_json=False):
    if is_json:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return json.dumps(random.sample(data, n), indent=2)
    else:
        df = pd.read_csv(file_path)
        return df.sample(n).to_json(orient='records', indent=2)

# For Customer Dataset Issue finder 

In [ ]:
summary_text_customers = format_summary_text(summarized_customers_data)
samples_text_customers = get_random_samples("../Data/customers_messy_data.json", n=10, is_json=True)
issues_report_customers = generate_data_issue_report("Customers", summary_text_customers, samples_text_customers)
issues_text_customers = issues_report_customers.issues
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", "B", 16)
pdf.cell(0, 10, "Customers Data Quality Report", ln=True, align="C")

pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Detected Data Issues found by LLM", ln=True)
pdf.set_font("Arial", "", 10)
for issue in issues_text_customers:
    pdf.multi_cell(0, 7, f"- {issue}")
pdf.ln(5)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Column-wise Summary generated by utility", ln=True)
pdf.set_font("Arial", "", 10)
for line in summary_text_customers.split('\n'):
    pdf.multi_cell(0, 7, line)

pdf.ln(5)
pdf.output("../reports/customers_data_quality_report.pdf")

issues=["Redundant columns: 'customer_id' (int) and 'cust_id' (str) both seem to identify the customer, but 'cust_id' has a specific format ('CUST_0001').", "Redundant columns: 'email' and 'email_address' both store email information, but they have different null percentages and potentially different values.", "Redundant columns: 'phone' and 'phone_number' both store phone number information, but they have different null percentages and potentially different formats.", "Redundant columns: 'zip_code' (float) and 'postal_code' (str) both store postal code information, but have different data types and null percentages.", "Redundant columns: 'registration_date' and 'reg_date' both store registration date information, but have different formats and null percentages.", "Inconsistent naming: 'customer_name' contains values that seem like usernames (e.g., 'henry.davis123', 'jane_doe', 'alice.johnson@email.com') while 'full_name' contains actual names. The sample records confirm name swapping 

''

# For Products dataset Issue finder 

In [36]:
summary_text_products = format_summary_text(summarized_products_data)
samples_text_products = get_random_samples("../Data/products_inconsistent_data.json", n=10, is_json=True)
issues_report_products = generate_data_issue_report("Products", summary_text_products, samples_text_products)
issues_text_products = issues_report_products.issues
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", "B", 16)
pdf.cell(0, 10, "Products Data Quality Report", ln=True, align="C")
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Detected Data Issues found by LLM", ln=True)
pdf.set_font("Arial", "", 10)
for issue in issues_text_products:
    pdf.multi_cell(0, 7, f"- {issue}")

pdf.ln(5)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Column-wise Summary generated by utility", ln=True)
pdf.set_font("Arial", "", 10)
for line in summary_text_products.split('\n'):
    pdf.multi_cell(0, 7, line)
pdf.ln(5)
pdf.output("../reports/products_data_quality_report.pdf")

''

# For Orders Dataset Issue finder

In [37]:
summary_text_orders = format_summary_text(summarized_orders_data)
samples_text_orders = get_random_samples("../Data/orders_unstructured_data.csv", n=10, is_json=False)
issues_report_orders = generate_data_issue_report("Orders", summary_text_orders, samples_text_orders)
issues_text_orders = issues_report_orders.issues
pdf = FPDF()
pdf.add_page()  
pdf.set_font("Arial", "B", 16)
pdf.cell(0, 10, "Orders Data Quality Report", ln=True, align="C")

pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Detected Data Issues found by LLM", ln=True)
pdf.set_font("Arial", "", 10)
for issue in issues_text_orders:
    pdf.multi_cell(0, 7, f"- {issue}")

pdf.ln(5)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Column-wise Summary generated using utility", ln=True)
pdf.set_font("Arial", "", 10)
for line in summary_text_orders.split('\n'):
    pdf.multi_cell(0, 7, line)
pdf.ln(5)
pdf.output("../reports/orders_data_quality_report.pdf")

''